## Create parquet files in GCS

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from snakeclef.utils import get_spark
import os
from pathlib import Path

spark = get_spark()
display(spark)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/07 17:09:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/07 17:09:30 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [3]:
# Get list of stored filed in cloud bucket
! gcloud storage ls gs://dsgt-clef-snakeclef-2024/data/parquet_files/

gs://dsgt-clef-snakeclef-2024/data/parquet_files/

gs://dsgt-clef-snakeclef-2024/data/parquet_files/:
gs://dsgt-clef-snakeclef-2024/data/parquet_files/
gs://dsgt-clef-snakeclef-2024/data/parquet_files/SnakeCLEF2023-train-large_size/
gs://dsgt-clef-snakeclef-2024/data/parquet_files/SnakeCLEF2023-train-medium_size/
gs://dsgt-clef-snakeclef-2024/data/parquet_files/SnakeCLEF2023-train-small_size/
gs://dsgt-clef-snakeclef-2024/data/parquet_files/acm_image_data_test_repartition/


In [4]:
# Get current directory
curr_dir = Path(os.getcwd())

# Change to the project directory to run the scripts
os.chdir(curr_dir.parents[1])

/home/mgustine/snakeclef-2024


In [5]:
import os


def process_dataset(dataset_name, meta_dataset_name, delete_dataset: bool = True):
    # Define the base directory
    base_dir = "/mnt/data"

    # Define the path for the script and Python script
    script_path = "./scripts/download_extract_dataset.sh"
    python_script_path = "./snakeclef/images_to_parquet.py"

    # Define the GCS paths
    gcs_path = f"gs://dsgt-clef-snakeclef-2024/raw/{dataset_name}.tar.gz"
    output_path = f"gs://dsgt-clef-snakeclef-2024/data/parquet_files/{dataset_name}"

    # Adjust the dataset_name for the --dataset-name parameter
    adjusted_dataset_name = (
        dataset_name.replace("train-", "")
        .replace("val-", "")
        .replace("pubtest", "pubtest")
    )

    # Download and extract the dataset
    os.system(f"{script_path} {gcs_path} {base_dir}")

    # Create parquet file
    os.system(
        f"python {python_script_path} --output-path {output_path} --dataset-name {adjusted_dataset_name} --meta-dataset-name {meta_dataset_name}"
    )

    if delete_dataset:
        # Delete dataset locally before loading the next one
        local_data_path = f"./data/{adjusted_dataset_name}"
        local_tar_path = f"./data/{dataset_name}.tar.gz"
        os.system(f"rm -rf {local_data_path}")
        os.system(f"rm -f {local_tar_path}")

### train small size

In [ ]:
# Define dataset names
data_name = "SnakeCLEF2023-train-small_size"
meta_name = "SnakeCLEF2023-TrainMetadata-iNat"

# Process the data
process_dataset(dataset_name=data_name, meta_dataset_name=meta_name)

### train medium size

In [ ]:
# Define dataset names
data_name = "SnakeCLEF2023-train-medium_size"
meta_name = "SnakeCLEF2023-TrainMetadata-iNat"

# Process the data
process_dataset(dataset_name=data_name, meta_dataset_name=meta_name)

### train large size

In [ ]:
# Define dataset names
data_name = "SnakeCLEF2023-train-large_size"
meta_name = "SnakeCLEF2023-TrainMetadata-iNat"

# Process the data
process_dataset(dataset_name=data_name, meta_dataset_name=meta_name)